In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import sqlalchemy
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

## settings

In [ ]:
data_folder = "mydirectory"
local_crs = 3006
plot = True
write_to_table = True

## Stockholm 2014 data

In [ ]:
path = os.path.join(data_folder, 'vehicles', 'STHLM', 'Trafikflode_Motorfordon.gpkg')
gdf = gpd.read_file(path, layer='Trafikflode_Motorfordon').to_crs(local_crs)
carflows_sthlm_2014 = gdf[gdf.Flödeskarta__version==2014]

if plot:
    carflows_sthlm_2014.plot(column='Totalflöde_alla_riktningar', cmap='Reds', categorical=True)

In [ ]:
main_col = 'Totalflöde_alla_riktningar'

In [ ]:
x = carflows_sthlm_2014[main_col]

In [ ]:
plt.hist(x, bins=50)
plt.show()

In [ ]:
if plot:
    plt.hist(x, bins=10**np.linspace(1, 6, 50))
    plt.xscale('log')
    plt.show()

In [ ]:
values = np.linspace(0, 500, 11)
for value in values:
    print('{}% of roads have ground truth flow value of {}'.format(round(100*(len(carflows_sthlm_2014[carflows_sthlm_2014[main_col]==value]) / len(carflows_sthlm_2014)), 2), value))

In [ ]:
if plot:
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(10,10))

    carflows_sthlm_2014[~(carflows_sthlm_2014[main_col].isin([50, 200, 300]))].plot(ax=axs, color='grey', linewidth=0.5)
    carflows_sthlm_2014[(carflows_sthlm_2014[main_col].isin([200, 300]))].plot(ax=axs, color='red', linewidth=0.5)
    carflows_sthlm_2014[(carflows_sthlm_2014[main_col].isin([50]))].plot(ax=axs, color='yellow', linewidth=0.5)

    axs.axis('off')

    handles = [
        mpatches.Patch(color='red', alpha=0.3, label='exactly 200 or 300 vehicles/hour'),
        mpatches.Patch(color='yellow', alpha=0.3, label='exactly 50 vehicles/hour'),
        mpatches.Patch(color='grey', alpha=0.3, label='Any other value')]
    axs.legend(handles=handles, loc='upper right')

    plt.show()

In [ ]:
carflows_sthlm_2014 = carflows_sthlm_2014[~carflows_sthlm_2014[main_col].isin([200,300])].copy()

In [ ]:
x = carflows_sthlm_2014[main_col]

if plot:
    plt.hist(x, bins=10**np.linspace(1, 6, 50))
    plt.xscale('log')
    plt.show()

In [ ]:
x = carflows_sthlm_2014[carflows_sthlm_2014[main_col]<300][main_col]

if plot:
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(20,5))
    plt.hist(x, bins=np.linspace(50, 500, 451))
    plt.show()

In [ ]:
carflows_sthlm_2014 = carflows_sthlm_2014[~carflows_sthlm_2014.Totalflöde_alla_riktningar.isin([50])].copy()

In [ ]:
carflows_sthlm_2014[main_col+'_normalized'] = carflows_sthlm_2014[main_col]/carflows_sthlm_2014[main_col].max()

## Stockholm 2024 data

In [ ]:
path = os.path.join(data_folder, 'vehicles', 'STHLM', 'Arstrafik.gpkg')
gdf = gpd.read_file(path, layer='Arstrafik').to_crs(local_crs)
carflows_sthlm_2024 = gdf[gdf.Datum_för_start_av_perioden==20240101]

carflows_sthlm_2024 = carflows_sthlm_2024[carflows_sthlm_2024.MAIN_ATTRIBUTE_DESCRIPTION.str.contains('Totalt')]
carflows_sthlm_2024.drop_duplicates('geometry', inplace=True)

main_col = 'Trafikflöde__fordon_dygn__alla_fordon'

if plot:
    carflows_sthlm_2024.plot(column=main_col, cmap='Reds', categorical=True)

In [ ]:
x = carflows_sthlm_2024[main_col]

In [ ]:
if plot:
    plt.hist(x, bins=50)
    plt.show()

In [ ]:
if plot:
    plt.hist(x, bins=10**np.linspace(1, 6, 50))
    plt.xscale('log')
    plt.show()

In [ ]:
carflows_sthlm_2024[main_col+'_normalized'] = carflows_sthlm_2024[main_col]/carflows_sthlm_2024[main_col].max()

## Gothenburg 2023/2024 data local roads

In [ ]:
path = os.path.join(data_folder, 'vehicles', 'GBG', 'Trafikmangdskatalogen.xlsx')
df = pd.read_excel(path)
df = df[~df['Mätplats-id'].isin([158,418])]   # insensible coordinates
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Mätplatskoordinat SWEREF991200 Long'], df['Mätplatskoordinat SWEREF991200 Lat']), crs=local_crs)
carflows_gbg_local_2023 = gdf[gdf['År']==2023].copy()
carflows_gbg_local_2024 = gdf[gdf['År']==2024].copy()

carflows_gbg_local_2023 = carflows_gbg_local_2023[carflows_gbg_local_2023.Mätsnittsnamn.isnull()]
carflows_gbg_local_2023.drop_duplicates('geometry', inplace=True)
carflows_gbg_local_2024 = carflows_gbg_local_2024[carflows_gbg_local_2024.Mätsnittsnamn.isnull()]
carflows_gbg_local_2024.drop_duplicates('geometry', inplace=True)

main_col = 'ÅDT'

if plot:
    fig, axs = plt.subplots(1, 2)
    carflows_gbg_local_2023.plot(ax=axs[0], column=main_col, cmap='Reds', categorical=True)
    carflows_gbg_local_2024.plot(ax=axs[1], column=main_col, cmap='Reds', categorical=True)
    plt.show()

In [ ]:
x1 = carflows_gbg_local_2023[main_col]
x2 = carflows_gbg_local_2024[main_col]

In [ ]:
if plot:
    plt.hist(x1, bins=50)
    plt.hist(x2, bins=50)

In [ ]:
if plot:
    plt.hist(x1, bins=10**np.linspace(1, 6, 50))
    plt.hist(x2, bins=10**np.linspace(1, 6, 50))
    plt.xscale('log')

In [ ]:
carflows_gbg_local_2023[main_col+'_normalized'] = carflows_gbg_local_2023[main_col]/carflows_gbg_local_2023[main_col].max()
carflows_gbg_local_2024[main_col+'_normalized'] = carflows_gbg_local_2024[main_col]/carflows_gbg_local_2024[main_col].max()

## Gothenburg 2024 data highways

In [ ]:
path = os.path.join(data_folder, 'vehicles', 'GBG', 'traffik_data_GBG_20241001_Geopackage_358047', 'traffik_data_GBG_20241001_358047.gpkg')
gdf = gpd.read_file(path, layer='TRAFIK_DK_O_105_Trafik').to_crs(local_crs)

carflows_gbg_highways_2023 = gdf[gdf['Matarsperiod']==202301].copy()
carflows_gbg_highways_2024 = gdf[gdf['Matarsperiod']==202401].copy()

if plot:
    fig, axs = plt.subplots(1, 2)
    carflows_gbg_highways_2023.plot(ax=axs[0], column='Adt_samtliga_fordon', cmap='Reds', categorical=True)
    carflows_gbg_highways_2024.plot(ax=axs[1], column='Adt_samtliga_fordon', cmap='Reds', categorical=True)
    plt.show()

In [ ]:
main_col = 'Adt_samtliga_fordon'

In [ ]:
x1 = carflows_gbg_highways_2023[main_col]
x2 = carflows_gbg_highways_2024[main_col]

In [ ]:
if plot:
    plt.hist(x1, bins=50)
    plt.hist(x2, bins=50)

In [ ]:
if plot:
    plt.hist(x1, bins=10**np.linspace(1, 6, 50))
    plt.hist(x2, bins=10**np.linspace(1, 6, 50))
    plt.xscale('log')

In [ ]:
carflows_gbg_highways_2023[main_col+'_normalized'] = carflows_gbg_highways_2023[main_col]/carflows_gbg_highways_2023[main_col].max()
carflows_gbg_highways_2024[main_col+'_normalized'] = carflows_gbg_highways_2024[main_col]/carflows_gbg_highways_2024[main_col].max()

## write data to table

In [ ]:
carflows_sthlm_2014.columns = carflows_sthlm_2014.columns.str.lower()
carflows_sthlm_2024.columns = carflows_sthlm_2024.columns.str.lower()
carflows_gbg_local_2023.columns = carflows_gbg_local_2023.columns.str.lower()
carflows_gbg_local_2024.columns = carflows_gbg_local_2024.columns.str.lower()
carflows_gbg_highways_2023.columns = carflows_gbg_highways_2023.columns.str.lower()
carflows_gbg_highways_2024.columns = carflows_gbg_highways_2024.columns.str.lower()

In [ ]:
# write to database
if write_to_table:

    url_flowsense = sqlalchemy.URL.create(
        "postgresql+psycopg", port=5432,
        host="host", database="database", username="username")
    engine_flowsense = sqlalchemy.create_engine(url_flowsense)

    carflows_sthlm_2014.to_postgis(
        name='flows_cars_sthlm_2014',
        con=engine_flowsense,
        schema='ground_truth',
        if_exists='replace',
        index=False)

    carflows_sthlm_2024.to_postgis(
        name='flows_cars_sthlm_2024',
        con=engine_flowsense,
        schema='ground_truth',
        if_exists='replace',
        index=False)

    carflows_gbg_local_2023.to_postgis(
        name='flows_cars_gbg_local_2023',
        con=engine_flowsense,
        schema='ground_truth',
        if_exists='replace',
        index=False)

    carflows_gbg_local_2024.to_postgis(
        name='flows_cars_gbg_local_2024',
        con=engine_flowsense,
        schema='ground_truth',
        if_exists='replace',
        index=False)

    carflows_gbg_highways_2023.to_postgis(
        name='flows_cars_gbg_highway_2023',
        con=engine_flowsense,
        schema='ground_truth',
        if_exists='replace',
        index=False)

    carflows_gbg_highways_2024.to_postgis(
        name='flows_cars_gbg_highway_2024',
        con=engine_flowsense,
        schema='ground_truth',
        if_exists='replace',
        index=False)